In [ ]:
from transformers import pipeline  #mask test

unmasker = pipeline('fill-mask', model='bert-base-multilingual-cased')

unmasker("ชนิดประโยคในภาษา [MASK] มีอะไรบ้าง")


In [5]:
from transformers import pipeline

classifier = pipeline("text-classification", model="./ecommerce_model", tokenizer="./ecommerce_model")

text = "มีสินค้าในสต๊อกไหม"
result = classifier(text)
print(result)

Device set to use cpu
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


[{'label': 'LABEL_1', 'score': 0.9994086027145386}]


In [ ]:
import pandas as pd

# Sample dataset for e-commerce chat classification
data = {
    "text": [
        "What is the price of iPhone?",
        "สินค้าหมดเมื่อไหร่จะมาอีก?",
        "I want to cancel my order.",
        "ต้องการเปลี่ยนที่อยู่จัดส่ง",
        "How can I track my order?",
        "ได้รับสินค้าผิดชิ้น",
        "The product is damaged.",
        "ขอคืนเงิน",
        "I want to give feedback.",
        "บริการลูกค้าแย่มาก"
    ],
    "label": [
        "Product Inquiry",
        "Product Inquiry",
        "Order Management",
        "Order Management",
        "Order Management",
        "Customer Support",
        "Customer Support",
        "Customer Support",
        "Feedback & Complaints",
        "Feedback & Complaints"
    ]
}

# Create DataFrame
df = pd.DataFrame(data)

# Save to CSV
file_path = "ecommerce_chat_dataset.csv"
df.to_csv(file_path, index=False)

file_path


In [ ]:
!pip install tensorflow-cpu
!pip install tf-keras
!pip install wandb

In [1]:
import wandb

# Log in to W&B using your API key
wandb.login(key="local-3be8233374a796b3fed2248de19c52b03220d770")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for 192.168.1.101:8080 to your netrc file: /home/meownani/.netrc
wandb: Currently logged in as: meownani to http://192.168.1.101:8080. Use `wandb login --relogin` to force relogin


True

In [4]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch
from datasets import Dataset
import wandb

# Log in to wandb using the API key (if needed)
wandb.login()  # Or wandb.login(key="your-api-key") if you want to specify the API key

# Load dataset
df = pd.read_csv("ecommerce_chat_dataset_thai_2000.csv")
labels = df['label'].unique()
label2id = {label: i for i, label in enumerate(labels)}
df['label'] = df['label'].map(label2id)

# Split data
train_texts, test_texts, train_labels, test_labels = train_test_split(df['text'], df['label'], test_size=0.2)

# Tokenization
model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch['text'], padding="max_length", truncation=True, max_length=128)

train_dataset = Dataset.from_dict({"text": train_texts.tolist(), "label": train_labels.tolist()}).map(tokenize, batched=True)
test_dataset = Dataset.from_dict({"text": test_texts.tolist(), "label": test_labels.tolist()}).map(tokenize, batched=True)

# Load Model
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(labels))

# Training Setup
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",  # This evaluates the model at the end of each epoch
    save_strategy="epoch",       # This saves the model at the end of each epoch (matches evaluation)
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train
trainer.train()

# Save Model
model.save_pretrained("./ecommerce_model")
tokenizer.save_pretrained("./ecommerce_model")

# Evaluate
trainer.evaluate()


Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/meownani/miniconda3/envs/mad_llm/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss
1,0.001000,0.000673
2,0.000500,0.000351
3,0.000400,0.000292


{'eval_loss': 0.0002920256520155817,
 'eval_runtime': 52.0117,
 'eval_samples_per_second': 7.691,
 'eval_steps_per_second': 0.961,
 'epoch': 3.0}

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [6]:
label2id 

{'Order Management': 0,
 'Product Inquiry': 1,
 'Feedback & Complaints': 2,
 'Customer Support': 3}

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [ ]:
import pandas as pd
from transformers import pipeline

# Load dataset
df = pd.read_csv("ecommerce_chat_dataset_thai_2000.csv")
# Initialize the classifier
classifier = pipeline("text-classification", model="./ecommerce_model", tokenizer="./ecommerce_model")
# Loop through the 'text' column
for text in df['text']:
    #print(text)

    # Loop through the 'text' column and classify the text
    df['result'] = df['text'].apply(lambda text: classifier(text)[0]['label'])  # Assuming you want the label

    # Optionally, save the updated DataFrame to a new CSV
    df.to_csv("ecommerce_chat_with_results.csv", index=False)

    # Show the updated DataFrame (optional)
    #print(df.head())

